In [1]:
# Import libraries

import numpy as np
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sns

# Pretty display for notebooks

%matplotlib inline


# Allows the use of display() for DataFrames
from IPython.display import display 

# Ignore the warnings
import warnings
warnings.filterwarnings('ignore')

# Load the dataset
# train = pd.read_csv("../asset/train.csv")
# test = pd.read_csv("../asset/test.csv")
weather = pd.read_csv("../weather_final.csv")
# key = pd.read_csv("../asset/key.csv")
# submission_example = pd.read_csv("../asset/sampleSubmission.csv")

# Success - Display the first record
# print("Train data : ", train.shape)
# print("Test  data : ", test.shape)
print("weather  data : ", weather.shape)
# print("key  data : ", key.shape)
# print("submission  data : ", submission_example.shape)

import pickle
import awesome_functions as cf
import datetime
from dateutil.parser import parse

# 원본을 유지하기 위해서 카피
# df_train = train.copy()
df_weather = weather.copy()
# df_key = key.copy()
# df_test = test.copy()

weather  data :  (19665, 23)


# date 컬럼 추가

In [8]:
# date타입을 parsing해서 year를 반환
def handle_date_Y(a):
    return parse(a).year

# date타입을 parsing해서 month를 반환
def handle_date_M(a):
    return parse(a).month

# date타입을 parsing해서 day를 반환
def handle_date_D(a):
    return parse(a).day

# date타입을 parsing해서 weekday를 반환
def handle_date_WeekDay(a):
    return parse(a).weekday()

In [9]:
df_weather['weekday'] = df_weather['date'].apply(handle_date_WeekDay)

# df_weather에 weekday 추가

In [11]:
df_weather.head()

,station_nbr,date,tmax,tmin,tavg,depart,dewpoint,wetbulb,heat,cool,...,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed,year,month,day,weekday
0,1,2012-01-01,52,31,41,11,36,40,24,0,...,0.05,29.78,29.92,3.6,20.0,4.6,2012,1,1,6
1,1,2012-01-02,50,31,40,10,26,35,25,0,...,0.01,29.44,29.62,9.8,24.0,10.3,2012,1,2,0
2,1,2012-01-03,32,11,21,-8,4,18,44,0,...,0.00,29.67,29.87,10.8,31.0,11.6,2012,1,3,1
3,1,2012-01-04,28,9,18,-11,-1,14,47,0,...,0.00,29.86,30.03,6.3,27.0,8.3,2012,1,4,2
4,1,2012-01-05,38,25,31,1,13,25,34,0,...,0.00,29.67,29.84,6.9,25.0,7.8,2012,1,5,3


# Add Holyday

In [13]:
import re
with open('holiday_names.txt') as file:  
    data = file.readlines()
    holidays = []
    BF = []
    months = ["", "Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
    for holiday in data:
        if re.findall("BlackFriday", holiday) != []:
            BF.append(holiday.split(" ")[:3])
        else:
            holidays.append(holiday.split(" ")[:3])
    for i in range(len(holidays)): 
        month_idx = months.index(holidays[i][1])
        if month_idx < 10:
            holidays[i][1] = "0" + str(month_idx)
        else:
            holidays[i][1] = str(month_idx)
    for z in range(len(holidays)): 
        if len(holidays[z][2]) == 1: 
            holidays[z][2] = "0" + holidays[z][2]
    for z in range(len(BF)) :
        BF[z][1] = "11"
        
        
# holiday 구분하기 
def isholiday(dt): 
    if dt.split("-") in BF :
        return "black friday"
    elif dt.split("-") in holidays :
        return "holiday"
    else :
        return "regular day"
    
df = df_weather.copy() # weather 데이터프레임

# holiday라는 컬럼추가. 위에서 보듯 세가지로만 구분되어있음.
# from_formular에서 C('holiday')사용하면 3가지 컬럼으로 바뀐다는 의미.
df['holiday'] = df['date'].apply(isholiday) 

In [15]:
df.shape

(19665, 25)

In [16]:
cf.isThereNoneData(df)

M Data over 60% : []
T Data over 60% : []
B Data over 60% : []


,Column,Row Count,Missing Data,M Data %,Trace Data,T Data %,Bar Data,B Data %
0,station_nbr,19665,0,0.0,0,0.0,0,0.00
1,date,19665,0,0.0,0,0.0,0,0.00
2,tmax,19665,0,0.0,0,0.0,0,0.00
3,tmin,19665,0,0.0,0,0.0,0,0.00
4,tavg,19665,0,0.0,0,0.0,0,0.00
5,depart,19665,0,0.0,0,0.0,0,0.00
6,dewpoint,19665,0,0.0,0,0.0,0,0.00
7,wetbulb,19665,0,0.0,0,0.0,0,0.00
8,heat,19665,0,0.0,0,0.0,0,0.00
9,cool,19665,0,0.0,0,0.0,0,0.00


In [18]:
cf.saveDataFrameToCsv(df, 'weather+holyday')